### NOTE : The below code uses Selenium to get reviews of a restaurant given a url. Since the time I made this, Zomato have changed their site so obviously the code won't work. You need to update the get_review_dataframe function to extract dataframe of reviews along with their ratings, rest of the code will work fine¶

In [11]:
import os
import time
import pandas as pd
import requests
import random
import spacy
from bs4 import BeautifulSoup as soup
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
import time
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
from nltk.tokenize import sent_tokenize
import re 
import numpy as np

class FoodItems():
    
    def __init__(self, url, output_dir):
        self.url = url
        self.output_dir = output_dir
    
    def get_review_dataframe(self, number = 100, popular=True): 
        """This function generates dataframe from url reviews with reviews as a column and rating as another column"""
        """Parameters :
           number  -> number of reviews you want the dataframe to be created from (more reviews means more time to extract)
           popular -> If all reviews are to be considered or only the popular ones. if popular = True, number doesn't matter"""
        """Output :
           It returns the generated dataframe as output"""
        browser = webdriver.Chrome(executable_path='/Users/rishabh/Downloads/chromedriver 2')
        browser.get(self.url)
        time.sleep(1)
        if popular == True:
            try:
                while True:
                    element = browser.find_element_by_xpath('//*[@id="reviews-container"]/div[1]/div[3]/div[2]/div/div/span[1]')
                    browser.execute_script("arguments[0].click();", element)
                    time.sleep(4)
            except NoSuchElementException as e:
                pass
        else:
            element = browser.find_element_by_xpath('//*[@id="selectors"]/a[2]')
            browser.execute_script("arguments[0].click();", element)
            time.sleep(10)
            c = 0
            try:
                while c<=number:
                    element = browser.find_element_by_xpath('//*[@id="reviews-container"]/div[1]/div[3]/div/div/div[2]/div[1]/span[1]')
                    browser.execute_script("arguments[0].click();", element)
                    c+=5
                    time.sleep(5)
            except NoSuchElementException as e:
                pass
        html = browser.page_source
        soup1 = soup(html)
        df = pd.DataFrame()
        text = []
        rating = []
        for items in soup1.findAll('div',{'class':'rev-text'}):
            text.append(items.text.strip('\nRated\xa0\n').strip(' ').replace('\n','. '))
            rating.append(float(items.div['aria-label'].strip('Rated ')))
        df['Text'] = text
        df['Rating'] = rating
        return df
    
    def get_food_dict(self, df):
        """This function generates dictionary with food items as keys and every review as value having that food item"""
        """Parameters:
           df -> This function takes generated df from reviews as input"""
        """Output :
           It returns the dictionary with food items as keys and every review as value having that food item"""
        nlp = spacy.load(self.output_dir)
        di = {}
        for reviews in df['Text']:
            for sen in sent_tokenize(reviews):
                doc = nlp(sen)
                for tokens in doc.ents:
                    if tokens.text in di:
                        li = di[tokens.text]
                        li.append(sen)
                        di[tokens.text] = li
                    else:
                        di[tokens.text] = [sen]
        return di
    
    def merge_similar(self,item,li):
        """This is a helper function for merging similar items in reviews"""
        send_li = []
        for items in process.extract(item,li):
            if items[1]>=90:
                send_li.append(items[0])
        return send_li
         
    
    def merge_similar_dict(self,di):
        """This function merges similar items extracted by NER model"""
        """Parameter :
           di -> dictionary of items as keys and reviews as values"""
        """Output :
           Return updated dictionary"""
        li = []
        for items in di:
            li.append(items)
        d = {}
        notin = []
        for i in di:
            if i in notin:
                continue
            k = self.merge_similar(i,li)
            notin.extend(k)
            ji = []
            for it in k:
                ji.extend(di[it])
            try:
                itt = max(k, key=len)
                d[itt] = ji 
            except:
                continue
        return d
        
    def get_numerical_rating(self, items, item_name):
        """This function is a helper function to generate rating for reviews having the format Number/Number 
           example : 4.5/5 or 4/5 etc."""
        rat = 0
        rating_regex = re.compile(r'[+-]?([0-9]*[.])?[0-9]+\/[+-]?([0-9]*[.])?[0-9]+')
        lh, _, rh = items.partition(item_name)
        rh = rh.split()[:2]
        rh = ' '.join(rh)
        mo = rating_regex.search(rh)
        if mo!= None:
            rat = mo.group().split('/')[0]
        return float(rat)
    def get_sentence_rating(self, items):
        """This function is a helper function to generate rating based on sentiment analysis from vader sentiment"""
        sid_obj = SentimentIntensityAnalyzer() 
        sentiment_dict = sid_obj.polarity_scores(items)
        z = (sentiment_dict['compound'] + 1)/4 * 10
        z = float("{0:.2f}".format(z))
        return z
    def get_ratings(self, li, item_name):
        """This function generates ratings for an item in dictionary"""
        l2 = []
        for items in li:
            rating = self.get_numerical_rating(items,item_name)
            if rating!=0:
                l2.append(rating)
            else:
                rating = self.get_sentence_rating(items)
                l2.append(rating)
        return (sum(l2)/len(l2))
    
    def generate_ratings(self,d):
        """This function generates ratings for every item"""
        d2 = {}
        for items in d:
            d2[items] = np.nan
        for items in d:
            d2[items] = self.get_ratings(d[items],items)
        return (sorted(d2.items(), key = lambda kv:(kv[1], kv[0]),reverse=True))  
    
    def final(self, number = 100, popular = True):
        """This function compiles all the functions together"""
        df = self.get_review_dataframe(number = number, popular = popular)
        di = self.get_food_dict(df = df)
        d = self.merge_similar_dict(di=di)
        output = self.generate_ratings(d=d)
        return output
        
def main_function(url,output_dir, number=100, popular=True):
    """Main function to be called to generated dictionary with items extracted and their ratings
    Parameters :
    url        ->   url of restaurant from zomato consisting of reviews
    output_dir ->   directory where custom NER model is saved
    number     ->   number of reviews to consider to be used for getting food items
    popular    ->   Whether to extract the reviews from only popular section or all reviews"""
    obj = FoodItems(url = url, output_dir = output_dir)
    final = obj.final(number = number, popular = popular)
    return final

In [12]:
### Uses URL of restaurant with reviews page selected as one parameter
### Uses directory where NER model is stored as other parameter
main_function(url = 'https://www.zomato.com/ncr/barbeque-nation-connaught-place-new-delhi/reviews', output_dir = '/Users/rishabh/Desktop/ProjectZomato/FoodNER')

/Applications/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Applications/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


[('mutton seekh kabab', 4.3),
 ('fish fingers', 4.3),
 ('tandoori chicken', 4.06),
 ('sweet corn', 4.06),
 ('mushrooms', 4.06),
 ('Candle light concept', 4.06),
 ('chicken, prawn', 3.9699999999999998),
 ('make', 3.96),
 ('delight', 3.96),
 ('CP', 3.93),
 ('Desserts', 3.9299999999999997),
 ('pineapple pastry', 3.8549999999999995),
 ('dal makhni', 3.82),
 ('chicken biryani', 3.82),
 ('Main course', 3.82),
 ('Food variety', 3.73),
 ('prawns', 3.7299999999999995),
 ('paneer tikka', 3.6999999999999997),
 ('red velvet pastry', 3.65),
 ('gulab jamun', 3.65),
 ('brownie', 3.65),
 ('Virgin Mary', 3.6),
 ('virgin mojito', 3.55),
 ('tandoori Platter', 3.55),
 ('sugar toast', 3.55),
 ('rabri', 3.55),
 ('pastries', 3.55),
 ('malai', 3.55),
 ('Noodles', 3.47),
 ('Barbeque Nation', 3.445),
 ('chicken tikka', 3.4),
 ('ice creams', 3.365),
 ('tamdoori snacks', 3.34),
 ('mint chutney', 3.34),
 ('grilled corn', 3.34),
 ('Grilled tandoori snacks', 3.34),
 ('strawberry.', 3.18),
 ('butter chicken', 3.16),


Outputs look decent enough, obviously with a model of high precision and some preprocessing on text being fed to the model, we can expect it to perform even better

#### Getting reviews of restaurant like this is not the best method obviously since I also suffered this -> zomato changed their site leaving my review extraction code worthless ! You can definitely switch to a better option like using their API to get the reviews as text